In [1]:
import requests
import json
import pprint

# Define Variables For Upload

In [12]:
instance_url = "https://vision-poc1.aus.stglabs.ibm.com/visual-inspection-v130-prod"
url_base = instance_url + "/api"
datasetid = "f6479060-2cad-4393-b6fa-23afbd55e8e5" #get this from the URL you're interacting with in the GUI
path_file_image = "./sampleimage.jpg" #path relative to our notebook
f = open(path_file_image, 'rb')
myfiles = [ ('files', f) ]
headers = {"X-Auth-Token": "INSERT_TOKEN_HERE"} # get from API key function or from vision-tools or /tokens API depending on version


# Upload Image and Metadata Separately

## Step 1: Upload the file and get its UUID
Note that you can upload multiple files if desired.

In [3]:
uploadresp = requests.post(url_base + "/datasets/" + datasetid + "/files",
                     headers=headers,
                     files=myfiles,
                     #data=json.dumps(data),
                     verify=False)
uploadresp

/Users/mchollin/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<Response [200]>

## Step 2: Parse the response, and find the UUID(s) of the files uploaded
Note that the API will respond with a resultList, and for each file uploaded pass an individual success or failure, and a mapping with the original source file to a UUID which may be used for further reference.

In [4]:
pprint.pprint((uploadresp.json()))

{'result': 'success',
 'resultList': [{'dataset_id': 'f6479060-2cad-4393-b6fa-23afbd55e8e5',
                 'id': '9cf08f1a-8eab-4150-a48a-bf52f24629b0',
                 'result': 'success',
                 'source_filepath': 'sampleimage.jpg'}]}


## Step 3: Save the UUID from the upload response

In [5]:
fileid = uploadresp.json()['resultList'][0]['id']
print(fileid)

9cf08f1a-8eab-4150-a48a-bf52f24629b0


## Step 4: Create a New Tag if Necessary
Labels are bound to a particular image by UUID. Each label is assigned a UUID at creation.

In [6]:
data = {"name":"examplelabel"}
resp = requests.post(url_base + "/datasets/" + datasetid + "/tags",
                     headers=headers,
                     data=json.dumps(data),
                     verify=False)
pprint.pprint((resp.json()))

{'dataset_tag_id': 'f6479060-2cad-4393-b6fa-23afbd55e8e5-examplelabel',
 'fault': 'Tag with same name already exists.',
 'result': 'fail'}


/Users/mchollin/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


## Step 4: Use the Tag and File UUID to save labels for the image

Set up JSON payload with valid tag id and bounding box information

In [7]:
tagid = resp.json()['dataset_tag_id']

labeldata = {"tag_id":tagid,
             "bndbox":{
                 "xmin":364,
                 "ymin":495,
                 "xmax":524,
                 "ymax":606}
            }

In [8]:
resp = requests.post(url_base + "/datasets/" + datasetid + "/files/" + fileid + "/object-labels",
                     headers=headers,
                     data=json.dumps(labeldata),
                     verify=False)

/Users/mchollin/.local/lib/python3.6/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [9]:
resp

<Response [200]>

In [10]:
pprint.pprint((resp.json()))

{'dataset_id': 'f6479060-2cad-4393-b6fa-23afbd55e8e5',
 'file_id': '9cf08f1a-8eab-4150-a48a-bf52f24629b0',
 'id': '7410c22a-f9de-45ac-a19a-468b48f23b0f',
 'name': 'examplelabel',
 'result': 'success',
 'tag_id': 'f6479060-2cad-4393-b6fa-23afbd55e8e5-examplelabel'}


This will result in a a new file (with the UUID shown above) being written to the data set with a single object label saved in it. To see this file, click on the URL below.

In [11]:
deeplink = instance_url + "#/datasets/"+ datasetid + "/label?imageId=" + fileid
print("Go to this URL in your browser to see the file you uploaded above!")
print(deeplink)

Go to this URL in your browser to see the file you uploaded above!
https://vision-poc1.aus.stglabs.ibm.com/visual-inspection-v130-prod#/datasets/f6479060-2cad-4393-b6fa-23afbd55e8e5/label?imageId=9cf08f1a-8eab-4150-a48a-bf52f24629b0
